In [2]:
#Grid de la segunda parte, en la que utilizamos ya la primera parte bien
#Esta va a ser la de las 44 variables con combined


#importamos paquetes
import sys
sys.path.append('Src/')
from data_modificado import * #hay funciones que estan cambiadas en este script para adaptralas a nuestro dataset
from train_2 import * #este hubo que modificar una linea tambien
from transfer_learning import * #hubo que modificart lo mismo que en train_2
from test_functions import *
from layers import *
from utils import *
from loss import *
from metric import *
from results import *
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
import itertools as it

#funciones
def read_df(
              metadata_names=['age','Temperature','Precipitation3Days'],
              random_state=42,
              otu_filename='../Datasets/otu_table_all_80.csv',
              metadata_filename='../Datasets/metadata_table_all_80.csv'):
    otu = pd.read_csv(otu_filename, index_col=0, header=None).T
    #print(otu.head())
    otu = otu.set_index('otuids')
    otu = otu.astype('int32')
    metadata = pd.read_csv(metadata_filename)
    #print(metadata.head())
    metadata = metadata.set_index('X.SampleID')
    metadata.head()
    domain = metadata[metadata_names]
    #if 'INBREDS' in metadata_names:
    #    domain = pd.concat([domain, pd.get_dummies(domain['INBREDS'], prefix='INBREDS')], axis=1)
    #    domain = domain.drop(['INBREDS'], axis=1)
    #elif 'Maize_Line' in metadata_names:
    #    domain = pd.concat([domain, pd.get_dummies(domain['Maize_Line'], prefix='Maize_Line')], axis=1)
    #    domain = domain.drop(['Maize_Line'], axis=1) 
    df = pd.concat([otu, domain], axis=1, sort=True, join='outer')
    #print(df.head())
    #data_microbioma = df[otu.columns].to_numpy(dtype=np.float32)
    #data_domain = df[domain.columns].to_numpy(dtype=np.float32)
    df_microbioma = df[otu.columns]
    df_domain = df[domain.columns]
    df_domain.head()
    df_microbioma_train, df_microbioma_no_train, df_domain_train, df_domain_no_train = \
        train_test_split(df_microbioma, df_domain, test_size=0.1, random_state=random_state)
    # Transfer learning subset
    df_microbioma_test, df_microbioma_transfer_learning, df_domain_test, df_domain_transfer_learning = \
        train_test_split(df_microbioma_no_train, df_domain_no_train, test_size=0.1, random_state=random_state)
    df_microbioma_transfer_learning_train, df_microbioma_transfer_learning_test, df_domain_transfer_learning_train, df_domain_transfer_learning_test = \
        train_test_split(df_microbioma_transfer_learning, df_domain_transfer_learning, test_size=0.3, random_state=random_state)
    
    return df_microbioma_train, df_microbioma_test, df_microbioma_transfer_learning_train, df_microbioma_transfer_learning_test, df_domain_train, df_domain_test, df_domain_transfer_learning_train, df_domain_transfer_learning_test, otu.columns, domain.columns
    #return df_microbioma_train, df_microbioma_test, df_microbioma_transfer_learning_train, df_microbioma_transfer_learning_test, df_domain_train, df_domain_test, df_domain_transfer_learning_train, df_domain_transfer_learning_test, otu.columns, domain.columns


def train_kfold_mod(model_fn, m_train, d_train, z_train, m_test, d_test, z_test,
                batch_size, epochs, train_callbacks):
    all_models = model_fn()
    model, encoder_bioma, encoder_domain, decoder_bioma = all_models
    metrics_prefix = None
    if encoder_bioma is not None and encoder_domain is not None:
        x_train = (m_train, d_train)
        y_train = (m_train, m_train, z_train)
        x_test = (m_test, d_test)
        y_test = (m_test, m_test, z_test)
    elif encoder_bioma is not None:
        x_train = m_train
        y_train = m_train
        x_test = m_test
        y_test = m_test
        metrics_prefix = 'bioma'
    elif encoder_domain is not None:
        x_train = d_train
        y_train = m_train
        x_test = d_test
        y_test = m_test
        metrics_prefix = 'domain'

    train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(5000).batch(
        batch_size)
    train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)
    val_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)
    val_dataset = val_dataset.prefetch(tf.data.experimental.AUTOTUNE)
    r = model.fit(train_dataset,
                  epochs=epochs,
                  validation_data=val_dataset,
                  callbacks=train_callbacks,
                  verbose=0)
    if metrics_prefix is not None:
        old_keys = r.history
        r.history = {}
        for k, v in old_keys.items():
            if k == 'loss' or k == 'val_loss':
                new_key = k
            elif k.startswith('val_'):
                new_key = 'val_{}_{}'.format(metrics_prefix, k[4:])
            else:
                new_key = '{}_{}'.format(metrics_prefix, k)
            r.history[new_key] = v
    del val_dataset
    del train_dataset
    del x_train
    del y_train
    del x_test
    del y_test
    return r, all_models

def train_2(model_fn,
          data_microbioma,
          data_domain,
          latent_space=10,
          folds=5,
          epochs=20,
          batch_size=128,
          learning_rate_scheduler=ExpDecayScheluder(),
          random_seed=347,
          verbose=0):
    data_zeros_latent = np.zeros((data_microbioma.shape[0], latent_space), dtype=data_microbioma.dtype)
    results = []
    models = []
    train_callbacks = [
        callbacks.EarlyStopping(monitor='val_loss', patience=epochs + 1, restore_best_weights=True)]
    if verbose >= 0:
        train_callbacks += [TqdmCallback(verbose=verbose)]
    if learning_rate_scheduler is not None:
        train_callbacks += [learning_rate_scheduler.make()]

    if folds <= 1:
        m_train, m_test = data_microbioma, data_microbioma
        d_train, d_test = data_domain, data_domain
        z_train, z_test = data_zeros_latent, data_zeros_latent
        tf.random.set_seed(random_seed)
        r, m = train_kfold(model_fn, m_train, d_train, z_train, m_test, d_test, z_test,
                           batch_size, epochs, train_callbacks)
        results.append(r)
        models.append(m)

    else: #EL PROBLEMA ESTA AQUI, QUE HACE FALTA UN 
        kf = KFold(n_splits=folds, random_state=random_seed, shuffle=True)
        tf.random.set_seed(random_seed)

        for train_index, test_index in kf.split(data_microbioma):
            m_train, m_test = data_microbioma[train_index], data_microbioma[test_index]
            #print(m_train)
            #d_train, d_test = data_domain[train_index], data_domain[test_index]
            if data_domain is None:
                d_train, d_test = None, None
            else:
                d_train, d_test = data_domain[train_index], data_domain[test_index]
            #print(d_train)
            #Esto de hacer el if else ha funcionado, pero no se si hace lo que debe bien
            z_train, z_test = data_zeros_latent[train_index], data_zeros_latent[test_index]
            r, m = train_kfold_mod(model_fn, m_train, d_train, z_train, m_test, d_test, z_test,
                               batch_size, epochs, train_callbacks)
            results.append(r)
            models.append(m)
    return results, models

def perform_experiment_2_mod(cv_folds, epochs, batch_size, learning_rate, optimizer,
                       learning_rate_scheduler, input_transform, output_transform,
                       reconstruction_loss, latent_space, layers,
                       activation, activation_latent,
                       data_microbioma_train, data_domain_train,
                       show_results=True, device='/CPU:0'): #Show results cambiado de False  aTrue
    if input_transform is not None:
        input_transform = input_transform()
    #----------    
    if output_transform is not None:
        output_transform = output_transform()
    #----------      
    if reconstruction_loss.__class__.__name__ == 'MakeLoss':
        reconstruction_loss = reconstruction_loss.make()
    else:
        reconstruction_loss = reconstruction_loss()
    domain_layers = [l // 16 for l in layers] ####que es esto???? Esto es para las capas del domain
    #print(domain_layers)
    bioma_autoencoder = " -> ".join(["b"] +
                                    [str(l) for l in layers] +
                                    [str(latent_space)] +
                                    [str(l) for l in reversed(layers)] +
                                    ["b"])
    #---------- 
    #esto solo se utiliza para el texto, es irrelevante para nuestro error
    if data_domain_train is not None:
        domain_autoencoder = " -> ".join(["d"] +
                                     [str(l) for l in domain_layers] +
                                     [str(latent_space)] +
                                     [str(l) for l in reversed(layers)] +
                                     ["b"])
        
    else: 
        domain_autoencoder = " "
    #---------- 
    #donde se usa domain autoencoder?
    in_transform_name = input_transform.__class__.__name__ if input_transform else "none"
    out_transform_name = output_transform.__class__.__name__ if output_transform else "none"
    lr_scheduler_text = learning_rate_scheduler[
        1] if learning_rate_scheduler is not None else "none"
    lr_text = learning_rate if learning_rate_scheduler is not None else "constant = {}".format(
        learning_rate)
    learning_rate_scheduler = learning_rate_scheduler[
        0] if learning_rate_scheduler is not None else None
    optimizer = optimizer(learning_rate=learning_rate)
    #---------- 
    experiment_parameters = [
        ("Input transform", in_transform_name),
        ("Output transform", out_transform_name),
        ("Reconstruction Loss", reconstruction_loss.__class__.__name__),
        ("Latent Space", latent_space),
        ("Bioma Autoencoder", bioma_autoencoder),
        ("Domain Autoencoder", domain_autoencoder),
        ("Activation Encoder", activation),
        ("Activation Decoder", activation),
        ("Activation Latent", activation_latent),
        ("CV folds", cv_folds),
        ("Epochs", epochs),
        ("Batch Size", batch_size),
        ("Learning Rate Scheduler", lr_scheduler_text),
        ("Learning Rate", lr_text),
        ("Optimizer", optimizer.__class__.__name__),
    ]
    #----------  
    if show_results:
        md_text = ""
        md_text += "| Parameter             | Value         |\n"
        md_text += "|:----------------------|:--------------|\n"
        for n, v in experiment_parameters:
            md_text += "| {} | {} |\n".format(n, v)

        display(Markdown(md_text))
    #------------
    def create_model(print_data=False):
        bioma_shape=data_microbioma_train.shape[1]
        
        if data_domain_train is not None:
            domain_shape=data_domain_train.shape[1]
            #print("data_domain_train!=None")
        else:
            domain_shape=None
            #print("data_domain_train==None")
        models = autoencoder(bioma_shape=bioma_shape,
                             #bioma_shape=717,
                             domain_shape=domain_shape,
                             output_shape=bioma_shape,
                             #output_shape=717,
                             latent_space=latent_space,
                             bioma_layers=layers, #Esto es lo de [512,316]
                             domain_layers=domain_layers, #Esto son cada una de las layers divididas por 16
                             input_transform=input_transform,
                             output_transform=output_transform,
                             activation_function_encoder=activation,
                             activation_function_decoder=activation,
                             activation_function_latent=activation_latent)
        #Entiendo analizando lo demas que aqui NO esta el error
        #la funcion autoencoder esta en model.py (es la unica funcion en ese script)
        
        model, encoder_bioma, encoder_domain, decoder_bioma = models

        if print_data:
            plot_models(model, encoder_bioma, encoder_domain, decoder_bioma)
        compile_train(model,
                      encoder_bioma=encoder_bioma,
                      encoder_domain=encoder_domain,
                      reconstruction_error=reconstruction_loss,
                      encoded_comparison_error=losses.MeanAbsoluteError(),
                      metrics=get_experiment_metrics(input_transform, output_transform),
                      optimizer=optimizer)
        
        #print("He acabado create_model :)")
        return model, encoder_bioma, encoder_domain, decoder_bioma
    #-----------
    create_model(print_data=False)
    #-----------
    #Esta en esta seccion el problema, en train_2
    #print(data_domain_train)
    #print(latent_space)
    with tf.device(device):
        results, models = train_2(create_model,
                                data_microbioma_train,
                                data_domain_train,
                                latent_space=latent_space,
                                folds=cv_folds,
                                epochs=epochs,
                                batch_size=batch_size,
                                learning_rate_scheduler=learning_rate_scheduler,
                                verbose=-1)
    #----------
    validation_results = print_results(results, show_results=show_results)
    if show_results:
        display(Markdown("*************"))

    return experiment_parameters + validation_results, models, results

__params:__
 - "activat_func":["softmax","sigmoid","relu","tanh"]
 - "activ_ouput":["softmax","sigmoid","relu","tanh"]
 - "learning_rate":[0.01,0.001]

In [ ]:
#Cargamos los datos
nombres_metadatos = ["KCAL","PROT","TFAT","CARB","MOIS","ALC","CAFF","THEO","SUGR","FIBE","CALC","IRON","MAGN","PHOS","POTA","SODI","ZINC","COPP","SELE","VC","VB1","VB2","NIAC","VB6","FOLA","VB12","VARA","RET","BCAR","ACAR","CRYP","LYCO","LZ","ATOC","VK","CHOLE","SFAT","MFAT","PFAT","VITD","CHOLN"]
#nombres_metadatos = ["PROT","TFAT","CARB","MOIS","ALC","CAFF","THEO","CALC","MAGN","POTA","ZINC","VC","VB1","VB6","VARA","ACAR","CRYP","LYCO","ATOC","VK","CHOLE","VITD"]
df_microbioma_train, df_microbioma_test, _, _, \
df_domain_train, df_domain_test, _, _, otu_columns, domain_columns = read_df(metadata_names=nombres_metadatos,otu_filename='resultados_ana/datos_otus_filtrados/otu_table_especies_80.csv',metadata_filename='resultados_ana/metadatos_nutrientes.csv')

data_microbioma_train = df_microbioma_train.to_numpy(dtype=np.float32)
data_microbioma_test = df_microbioma_test.to_numpy(dtype=np.float32)
data_domain_train = df_domain_train.to_numpy(dtype=np.float32)
data_domain_test = df_domain_test.to_numpy(dtype=np.float32)

#Preparamos las combinaciones pertinentes (5 mejores)
combinations = [[100,64,0.01,optimizers.Adam,15,[512,256],"tanh","tanh"],\
                [100,64,0.01,optimizers.Adam,15,[512,256],"tanh","softmax"],\
                [100,64,0.001,optimizers.Adam,15,[512,256],"tanh","tanh"],\
                [100,64,0.01,optimizers.Adam,10,[512,256],"tanh","softmax"],\
                [100,64,0.001,optimizers.Adam,15,[512,256,128],"tanh","tanh"]]

### 0.001, tanh, tanh

In [3]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='tanh')(in_layer)
        net = layers.Dense(64, activation='tanh')(net)
        net = layers.Dense(32, activation='tanh')(net)
        net = layers.Dense(16, activation='tanh')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation=None)(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4615940451622009 | 0.4615940451622009 | 0.4615940451622009 |
| pearson_corr | 0.6292402148246765 | 0.6292402148246765 | 0.6292402148246765 |
| jensen_shannon_divergence | 1.2567819356918335 | 1.2567819356918335 | 1.2567819356918335 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4661763906478882 | 0.4661763906478882 | 0.4661763906478882 |
| pearson_corr | 0.6502968072891235 | 0.6502968072891235 | 0.6502968072891235 |
| jensen_shannon_divergence | 1.2408562898635864 | 1.2408562898635864 | 1.2408562898635864 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4814591109752655 | 0.4814591109752655 | 0.4814591109752655 |
| pearson_corr | 0.6143974661827087 | 0.6143974661827087 | 0.6143974661827087 |
| jensen_shannon_divergence | 1.2916396856307983 | 1.2916396856307983 | 1.2916396856307983 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4580644369125366 | 0.4580644369125366 | 0.4580644369125366 |
| pearson_corr | 0.6617221236228943 | 0.6617221236228943 | 0.6617221236228943 |
| jensen_shannon_divergence | 1.17253577709198 | 1.17253577709198 | 1.17253577709198 |


----------------------------



### 0.001, tanh, softmax

In [4]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='tanh')(in_layer)
        net = layers.Dense(64, activation='tanh')(net)
        net = layers.Dense(32, activation='tanh')(net)
        net = layers.Dense(16, activation='tanh')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='softmax')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7979598641395569 | 0.7979598641395569 | 0.7979598641395569 |
| pearson_corr | 0.13567163050174713 | 0.13567163050174713 | 0.13567163050174713 |
| jensen_shannon_divergence | 4.317145824432373 | 4.317145824432373 | 4.317145824432373 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5114601850509644 | 0.5114601850509644 | 0.5114601850509644 |
| pearson_corr | 0.5781038999557495 | 0.5781038999557495 | 0.5781038999557495 |
| jensen_shannon_divergence | 1.4413093328475952 | 1.4413093328475952 | 1.4413093328475952 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5169218182563782 | 0.5169218182563782 | 0.5169218182563782 |
| pearson_corr | 0.5531112551689148 | 0.5531112551689148 | 0.5531112551689148 |
| jensen_shannon_divergence | 1.360981822013855 | 1.360981822013855 | 1.360981822013855 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6906418204307556 | 0.6906418204307556 | 0.6906418204307556 |
| pearson_corr | 0.47374534606933594 | 0.47374534606933594 | 0.47374534606933594 |
| jensen_shannon_divergence | 2.5578229427337646 | 2.5578229427337646 | 2.5578229427337646 |


----------------------------



### 0.001, tanh, sigmoid

In [10]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='tanh')(in_layer)
        net = layers.Dense(64, activation='tanh')(net)
        net = layers.Dense(32, activation='tanh')(net)
        net = layers.Dense(16, activation='tanh')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='sigmoid')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7901312708854675 | 0.7901312708854675 | 0.7901312708854675 |
| pearson_corr | 0.1154412180185318 | 0.1154412180185318 | 0.1154412180185318 |
| jensen_shannon_divergence | 4.429524898529053 | 4.429524898529053 | 4.429524898529053 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.46900588274002075 | 0.46900588274002075 | 0.46900588274002075 |
| pearson_corr | 0.6515414118766785 | 0.6515414118766785 | 0.6515414118766785 |
| jensen_shannon_divergence | 1.2248607873916626 | 1.2248607873916626 | 1.2248607873916626 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4713519811630249 | 0.4713519811630249 | 0.4713519811630249 |
| pearson_corr | 0.6476747393608093 | 0.6476747393608093 | 0.6476747393608093 |
| jensen_shannon_divergence | 1.1838138103485107 | 1.1838138103485107 | 1.1838138103485107 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7112184166908264 | 0.7112184166908264 | 0.7112184166908264 |
| pearson_corr | 0.4635128974914551 | 0.4635128974914551 | 0.4635128974914551 |
| jensen_shannon_divergence | 2.7780230045318604 | 2.7780230045318604 | 2.7780230045318604 |


----------------------------



### 0.001, tanh, relu

In [37]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='tanh')(in_layer)
        net = layers.Dense(64, activation='tanh')(net)
        net = layers.Dense(32, activation='tanh')(net)
        net = layers.Dense(16, activation='tanh')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='relu')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8169180750846863 | 0.8169180750846863 | 0.8169180750846863 |
| pearson_corr | 0.09516912698745728 | 0.09516912698745728 | 0.09516912698745728 |
| jensen_shannon_divergence | 4.637785911560059 | 4.637785911560059 | 4.637785911560059 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6119848489761353 | 0.6119848489761353 | 0.6119848489761353 |
| pearson_corr | 0.40699321031570435 | 0.40699321031570435 | 0.40699321031570435 |
| jensen_shannon_divergence | 1.965350866317749 | 1.965350866317749 | 1.965350866317749 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6275060176849365 | 0.6275060176849365 | 0.6275060176849365 |
| pearson_corr | 0.3920269310474396 | 0.3920269310474396 | 0.3920269310474396 |
| jensen_shannon_divergence | 2.0298898220062256 | 2.0298898220062256 | 2.0298898220062256 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.70380038022995 | 0.70380038022995 | 0.70380038022995 |
| pearson_corr | 0.46793997287750244 | 0.46793997287750244 | 0.46793997287750244 |
| jensen_shannon_divergence | 2.8022282123565674 | 2.8022282123565674 | 2.8022282123565674 |


----------------------------



### 0.001, softmax, tanh

In [6]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
  
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='softmax')(in_layer)
        net = layers.Dense(64, activation='softmax')(net)
        net = layers.Dense(32, activation='softmax')(net)
        net = layers.Dense(16, activation='softmax')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='tanh')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4639345407485962 | 0.4639345407485962 | 0.4639345407485962 |
| pearson_corr | 0.6380274295806885 | 0.6380274295806885 | 0.6380274295806885 |
| jensen_shannon_divergence | 1.2071113586425781 | 1.2071113586425781 | 1.2071113586425781 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.47478607296943665 | 0.47478607296943665 | 0.47478607296943665 |
| pearson_corr | 0.6394025683403015 | 0.6394025683403015 | 0.6394025683403015 |
| jensen_shannon_divergence | 1.2521212100982666 | 1.2521212100982666 | 1.2521212100982666 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.47592994570732117 | 0.47592994570732117 | 0.47592994570732117 |
| pearson_corr | 0.6348063349723816 | 0.6348063349723816 | 0.6348063349723816 |
| jensen_shannon_divergence | 1.2302792072296143 | 1.2302792072296143 | 1.2302792072296143 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.46743056178092957 | 0.46743056178092957 | 0.46743056178092957 |
| pearson_corr | 0.6556490659713745 | 0.6556490659713745 | 0.6556490659713745 |
| jensen_shannon_divergence | 1.218557596206665 | 1.218557596206665 | 1.218557596206665 |


----------------------------



### 0.001, softmax, softmax

In [7]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')

    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='softmax')(in_layer)
        net = layers.Dense(64, activation='softmax')(net)
        net = layers.Dense(32, activation='softmax')(net)
        net = layers.Dense(16, activation='softmax')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='softmax')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8107948899269104 | 0.8107948899269104 | 0.8107948899269104 |
| pearson_corr | 0.09773276746273041 | 0.09773276746273041 | 0.09773276746273041 |
| jensen_shannon_divergence | 4.684149742126465 | 4.684149742126465 | 4.684149742126465 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.47459205985069275 | 0.47459205985069275 | 0.47459205985069275 |
| pearson_corr | 0.6376132369041443 | 0.6376132369041443 | 0.6376132369041443 |
| jensen_shannon_divergence | 1.2483214139938354 | 1.2483214139938354 | 1.2483214139938354 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.47675371170043945 | 0.47675371170043945 | 0.47675371170043945 |
| pearson_corr | 0.6328918933868408 | 0.6328918933868408 | 0.6328918933868408 |
| jensen_shannon_divergence | 1.2285188436508179 | 1.2285188436508179 | 1.2285188436508179 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6927666068077087 | 0.6927666068077087 | 0.6927666068077087 |
| pearson_corr | 0.4717312753200531 | 0.4717312753200531 | 0.4717312753200531 |
| jensen_shannon_divergence | 2.586958408355713 | 2.586958408355713 | 2.586958408355713 |


----------------------------



### 0.001, softmax, sigmoid

In [8]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')

    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='softmax')(in_layer)
        net = layers.Dense(64, activation='softmax')(net)
        net = layers.Dense(32, activation='softmax')(net)
        net = layers.Dense(16, activation='softmax')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='sigmoid')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7619848847389221 | 0.7619848847389221 | 0.7619848847389221 |
| pearson_corr | 0.14676913619041443 | 0.14676913619041443 | 0.14676913619041443 |
| jensen_shannon_divergence | 4.135383129119873 | 4.135383129119873 | 4.135383129119873 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4828351140022278 | 0.4828351140022278 | 0.4828351140022278 |
| pearson_corr | 0.6286951303482056 | 0.6286951303482056 | 0.6286951303482056 |
| jensen_shannon_divergence | 1.2594006061553955 | 1.2594006061553955 | 1.2594006061553955 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4773045480251312 | 0.4773045480251312 | 0.4773045480251312 |
| pearson_corr | 0.6324997544288635 | 0.6324997544288635 | 0.6324997544288635 |
| jensen_shannon_divergence | 1.2294164896011353 | 1.2294164896011353 | 1.2294164896011353 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.72359699010849 | 0.72359699010849 | 0.72359699010849 |
| pearson_corr | 0.4316120445728302 | 0.4316120445728302 | 0.4316120445728302 |
| jensen_shannon_divergence | 3.0692853927612305 | 3.0692853927612305 | 3.0692853927612305 |


----------------------------



### 0.001, softmax, relu

In [9]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')


    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='softmax')(in_layer)
        net = layers.Dense(64, activation='softmax')(net)
        net = layers.Dense(32, activation='softmax')(net)
        net = layers.Dense(16, activation='softmax')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='relu')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8004556894302368 | 0.8004556894302368 | 0.8004556894302368 |
| pearson_corr | 0.10578623414039612 | 0.10578623414039612 | 0.10578623414039612 |
| jensen_shannon_divergence | 4.541911602020264 | 4.541911602020264 | 4.541911602020264 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5370185375213623 | 0.5370185375213623 | 0.5370185375213623 |
| pearson_corr | 0.5246917605400085 | 0.5246917605400085 | 0.5246917605400085 |
| jensen_shannon_divergence | 1.5357177257537842 | 1.5357177257537842 | 1.5357177257537842 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6929323673248291 | 0.6929323673248291 | 0.6929323673248291 |
| pearson_corr | 0.2540462911128998 | 0.2540462911128998 | 0.2540462911128998 |
| jensen_shannon_divergence | 2.6215293407440186 | 2.6215293407440186 | 2.6215293407440186 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7669365406036377 | 0.7669365406036377 | 0.7669365406036377 |
| pearson_corr | 0.10595495998859406 | 0.10595495998859406 | 0.10595495998859406 |
| jensen_shannon_divergence | 3.164177656173706 | 3.164177656173706 | 3.164177656173706 |


----------------------------



### 0.001, sigmoid, tanh

In [11]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')

    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='sigmoid')(in_layer)
        net = layers.Dense(64, activation='sigmoid')(net)
        net = layers.Dense(32, activation='sigmoid')(net)
        net = layers.Dense(16, activation='sigmoid')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='tanh')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4567837715148926 | 0.4567837715148926 | 0.4567837715148926 |
| pearson_corr | 0.6543636322021484 | 0.6543636322021484 | 0.6543636322021484 |
| jensen_shannon_divergence | 1.1668720245361328 | 1.1668720245361328 | 1.1668720245361328 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.483720988035202 | 0.483720988035202 | 0.483720988035202 |
| pearson_corr | 0.6353670358657837 | 0.6353670358657837 | 0.6353670358657837 |
| jensen_shannon_divergence | 1.2813714742660522 | 1.2813714742660522 | 1.2813714742660522 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4759332239627838 | 0.4759332239627838 | 0.4759332239627838 |
| pearson_corr | 0.6531193256378174 | 0.6531193256378174 | 0.6531193256378174 |
| jensen_shannon_divergence | 1.2045745849609375 | 1.2045745849609375 | 1.2045745849609375 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.45711249113082886 | 0.45711249113082886 | 0.45711249113082886 |
| pearson_corr | 0.6660574674606323 | 0.6660574674606323 | 0.6660574674606323 |
| jensen_shannon_divergence | 1.1803325414657593 | 1.1803325414657593 | 1.1803325414657593 |


----------------------------



### 0.001, sigmoid, softmax

In [12]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')

    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='sigmoid')(in_layer)
        net = layers.Dense(64, activation='sigmoid')(net)
        net = layers.Dense(32, activation='sigmoid')(net)
        net = layers.Dense(16, activation='sigmoid')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='softmax')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8156790137290955 | 0.8156790137290955 | 0.8156790137290955 |
| pearson_corr | 0.09388229995965958 | 0.09388229995965958 | 0.09388229995965958 |
| jensen_shannon_divergence | 4.740387439727783 | 4.740387439727783 | 4.740387439727783 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.468555748462677 | 0.468555748462677 | 0.468555748462677 |
| pearson_corr | 0.6449911594390869 | 0.6449911594390869 | 0.6449911594390869 |
| jensen_shannon_divergence | 1.2399178743362427 | 1.2399178743362427 | 1.2399178743362427 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.47216370701789856 | 0.47216370701789856 | 0.47216370701789856 |
| pearson_corr | 0.6468136310577393 | 0.6468136310577393 | 0.6468136310577393 |
| jensen_shannon_divergence | 1.2127269506454468 | 1.2127269506454468 | 1.2127269506454468 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6909114718437195 | 0.6909114718437195 | 0.6909114718437195 |
| pearson_corr | 0.4749881327152252 | 0.4749881327152252 | 0.4749881327152252 |
| jensen_shannon_divergence | 2.5527286529541016 | 2.5527286529541016 | 2.5527286529541016 |


----------------------------



### 0.001, sigmoid, sigmoid

In [13]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')

    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='sigmoid')(in_layer)
        net = layers.Dense(64, activation='sigmoid')(net)
        net = layers.Dense(32, activation='sigmoid')(net)
        net = layers.Dense(16, activation='sigmoid')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='sigmoid')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7926605939865112 | 0.7926605939865112 | 0.7926605939865112 |
| pearson_corr | 0.11306102573871613 | 0.11306102573871613 | 0.11306102573871613 |
| jensen_shannon_divergence | 4.459813117980957 | 4.459813117980957 | 4.459813117980957 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4744028151035309 | 0.4744028151035309 | 0.4744028151035309 |
| pearson_corr | 0.6378796100616455 | 0.6378796100616455 | 0.6378796100616455 |
| jensen_shannon_divergence | 1.2472792863845825 | 1.2472792863845825 | 1.2472792863845825 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4768528342247009 | 0.4768528342247009 | 0.4768528342247009 |
| pearson_corr | 0.6328268647193909 | 0.6328268647193909 | 0.6328268647193909 |
| jensen_shannon_divergence | 1.2282215356826782 | 1.2282215356826782 | 1.2282215356826782 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7115454077720642 | 0.7115454077720642 | 0.7115454077720642 |
| pearson_corr | 0.46221303939819336 | 0.46221303939819336 | 0.46221303939819336 |
| jensen_shannon_divergence | 2.793532133102417 | 2.793532133102417 | 2.793532133102417 |


----------------------------



### 0.001, sigmoid, relu

In [14]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='sigmoid')(in_layer)
        net = layers.Dense(64, activation='sigmoid')(net)
        net = layers.Dense(32, activation='sigmoid')(net)
        net = layers.Dense(16, activation='sigmoid')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='relu')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7979992032051086 | 0.7979992032051086 | 0.7979992032051086 |
| pearson_corr | 0.10924278199672699 | 0.10924278199672699 | 0.10924278199672699 |
| jensen_shannon_divergence | 4.436927318572998 | 4.436927318572998 | 4.436927318572998 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5691888928413391 | 0.5691888928413391 | 0.5691888928413391 |
| pearson_corr | 0.49486294388771057 | 0.49486294388771057 | 0.49486294388771057 |
| jensen_shannon_divergence | 1.5556151866912842 | 1.5556151866912842 | 1.5556151866912842 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7100187540054321 | 0.7100187540054321 | 0.7100187540054321 |
| pearson_corr | 0.23355114459991455 | 0.23355114459991455 | 0.23355114459991455 |
| jensen_shannon_divergence | 2.811079740524292 | 2.811079740524292 | 2.811079740524292 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7547920942306519 | 0.7547920942306519 | 0.7547920942306519 |
| pearson_corr | 0.11766961216926575 | 0.11766961216926575 | 0.11766961216926575 |
| jensen_shannon_divergence | 3.039285898208618 | 3.039285898208618 | 3.039285898208618 |


----------------------------



### 0.001, relu, tanh

In [15]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='relu')(in_layer)
        net = layers.Dense(64, activation='relu')(net)
        net = layers.Dense(32, activation='relu')(net)
        net = layers.Dense(16, activation='relu')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='tanh')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.9789464473724365 | 0.9789464473724365 | 0.9789464473724365 |
| pearson_corr | 0.0177798829972744 | 0.0177798829972744 | 0.0177798829972744 |
| jensen_shannon_divergence | 11.925947189331055 | 11.925947189331055 | 11.925947189331055 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.9601624011993408 | 0.9601624011993408 | 0.9601624011993408 |
| pearson_corr | 0.0011071580229327083 | 0.0011071580229327083 | 0.0011071580229327083 |
| jensen_shannon_divergence | 12.869157791137695 | 12.869157791137695 | 12.869157791137695 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.9954437613487244 | 0.9954437613487244 | 0.9954437613487244 |
| pearson_corr | -0.0011861547827720642 | -0.0011861547827720642 | -0.0011861547827720642 |
| jensen_shannon_divergence | 31.05455207824707 | 31.05455207824707 | 31.05455207824707 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.9085354804992676 | 0.9085354804992676 | 0.9085354804992676 |
| pearson_corr | 0.062212005257606506 | 0.062212005257606506 | 0.062212005257606506 |
| jensen_shannon_divergence | 5.947768688201904 | 5.947768688201904 | 5.947768688201904 |


----------------------------



### 0.001, relu, softmax

In [16]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='relu')(in_layer)
        net = layers.Dense(64, activation='relu')(net)
        net = layers.Dense(32, activation='relu')(net)
        net = layers.Dense(16, activation='relu')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='softmax')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.9336927533149719 | 0.9336927533149719 | 0.9336927533149719 |
| pearson_corr | 0.03116733767092228 | 0.03116733767092228 | 0.03116733767092228 |
| jensen_shannon_divergence | 6.383246421813965 | 6.383246421813965 | 6.383246421813965 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.9554868936538696 | 0.9554868936538696 | 0.9554868936538696 |
| pearson_corr | 0.035315461456775665 | 0.035315461456775665 | 0.035315461456775665 |
| jensen_shannon_divergence | 7.905366897583008 | 7.905366897583008 | 7.905366897583008 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.9882022142410278 | 0.9882022142410278 | 0.9882022142410278 |
| pearson_corr | -0.005316429305821657 | -0.005316429305821657 | -0.005316429305821657 |
| jensen_shannon_divergence | 11.23586368560791 | 11.23586368560791 | 11.23586368560791 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8294870853424072 | 0.8294870853424072 | 0.8294870853424072 |
| pearson_corr | 0.09795175492763519 | 0.09795175492763519 | 0.09795175492763519 |
| jensen_shannon_divergence | 4.186568260192871 | 4.186568260192871 | 4.186568260192871 |


----------------------------



### 0.001, relu, sigmoid

In [17]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='relu')(in_layer)
        net = layers.Dense(64, activation='relu')(net)
        net = layers.Dense(32, activation='relu')(net)
        net = layers.Dense(16, activation='relu')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='sigmoid')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.9406698346138 | 0.9406698346138 | 0.9406698346138 |
| pearson_corr | 0.01040564849972725 | 0.01040564849972725 | 0.01040564849972725 |
| jensen_shannon_divergence | 6.592992782592773 | 6.592992782592773 | 6.592992782592773 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.9935759902000427 | 0.9935759902000427 | 0.9935759902000427 |
| pearson_corr | 0.007883450947701931 | 0.007883450947701931 | 0.007883450947701931 |
| jensen_shannon_divergence | 14.18040943145752 | 14.18040943145752 | 14.18040943145752 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.9963968992233276 | 0.9963968992233276 | 0.9963968992233276 |
| pearson_corr | 0.0027234728913754225 | 0.0027234728913754225 | 0.0027234728913754225 |
| jensen_shannon_divergence | 18.610271453857422 | 18.610271453857422 | 18.610271453857422 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8974078297615051 | 0.8974078297615051 | 0.8974078297615051 |
| pearson_corr | 0.15039697289466858 | 0.15039697289466858 | 0.15039697289466858 |
| jensen_shannon_divergence | 5.981939315795898 | 5.981939315795898 | 5.981939315795898 |


----------------------------



### 0.001, relu, relu

In [18]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='relu')(in_layer)
        net = layers.Dense(64, activation='relu')(net)
        net = layers.Dense(32, activation='relu')(net)
        net = layers.Dense(16, activation='relu')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='relu')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.001), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6169271469116211 | 0.6169271469116211 | 0.6169271469116211 |
| pearson_corr | 0.40900689363479614 | 0.40900689363479614 | 0.40900689363479614 |
| jensen_shannon_divergence | 2.392487049102783 | 2.392487049102783 | 2.392487049102783 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5481650829315186 | 0.5481650829315186 | 0.5481650829315186 |
| pearson_corr | 0.5755901336669922 | 0.5755901336669922 | 0.5755901336669922 |
| jensen_shannon_divergence | 2.688617467880249 | 2.688617467880249 | 2.688617467880249 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5376953482627869 | 0.5376953482627869 | 0.5376953482627869 |
| pearson_corr | 0.5629035234451294 | 0.5629035234451294 | 0.5629035234451294 |
| jensen_shannon_divergence | 1.679012417793274 | 1.679012417793274 | 1.679012417793274 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6988839507102966 | 0.6988839507102966 | 0.6988839507102966 |
| pearson_corr | 0.33759161829948425 | 0.33759161829948425 | 0.33759161829948425 |
| jensen_shannon_divergence | 2.7000129222869873 | 2.7000129222869873 | 2.7000129222869873 |


----------------------------



---------------------------
### 0.01, tanh, tanh

In [19]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='tanh')(in_layer)
        net = layers.Dense(64, activation='tanh')(net)
        net = layers.Dense(32, activation='tanh')(net)
        net = layers.Dense(16, activation='tanh')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='tanh')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4921734035015106 | 0.4921734035015106 | 0.4921734035015106 |
| pearson_corr | 0.6018792986869812 | 0.6018792986869812 | 0.6018792986869812 |
| jensen_shannon_divergence | 1.3725441694259644 | 1.3725441694259644 | 1.3725441694259644 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5988902449607849 | 0.5988902449607849 | 0.5988902449607849 |
| pearson_corr | 0.4058154225349426 | 0.4058154225349426 | 0.4058154225349426 |
| jensen_shannon_divergence | 2.0930824279785156 | 2.0930824279785156 | 2.0930824279785156 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5722740292549133 | 0.5722740292549133 | 0.5722740292549133 |
| pearson_corr | 0.4930988550186157 | 0.4930988550186157 | 0.4930988550186157 |
| jensen_shannon_divergence | 1.8408020734786987 | 1.8408020734786987 | 1.8408020734786987 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4998202621936798 | 0.4998202621936798 | 0.4998202621936798 |
| pearson_corr | 0.6154140830039978 | 0.6154140830039978 | 0.6154140830039978 |
| jensen_shannon_divergence | 1.3712679147720337 | 1.3712679147720337 | 1.3712679147720337 |


----------------------------



### 0.01, tanh, softmax

In [20]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='tanh')(in_layer)
        net = layers.Dense(64, activation='tanh')(net)
        net = layers.Dense(32, activation='tanh')(net)
        net = layers.Dense(16, activation='tanh')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='softmax')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8071914911270142 | 0.8071914911270142 | 0.8071914911270142 |
| pearson_corr | 0.10046995431184769 | 0.10046995431184769 | 0.10046995431184769 |
| jensen_shannon_divergence | 4.641350746154785 | 4.641350746154785 | 4.641350746154785 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4817841351032257 | 0.4817841351032257 | 0.4817841351032257 |
| pearson_corr | 0.634000837802887 | 0.634000837802887 | 0.634000837802887 |
| jensen_shannon_divergence | 1.2864137887954712 | 1.2864137887954712 | 1.2864137887954712 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4734899699687958 | 0.4734899699687958 | 0.4734899699687958 |
| pearson_corr | 0.6393817067146301 | 0.6393817067146301 | 0.6393817067146301 |
| jensen_shannon_divergence | 1.2552577257156372 | 1.2552577257156372 | 1.2552577257156372 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.692314863204956 | 0.692314863204956 | 0.692314863204956 |
| pearson_corr | 0.47280657291412354 | 0.47280657291412354 | 0.47280657291412354 |
| jensen_shannon_divergence | 2.587129592895508 | 2.587129592895508 | 2.587129592895508 |


----------------------------



### 0.01, tanh, sigmoid

In [22]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='tanh')(in_layer)
        net = layers.Dense(64, activation='tanh')(net)
        net = layers.Dense(32, activation='tanh')(net)
        net = layers.Dense(16, activation='tanh')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='sigmoid')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7990065217018127 | 0.7990065217018127 | 0.7990065217018127 |
| pearson_corr | 0.10714982450008392 | 0.10714982450008392 | 0.10714982450008392 |
| jensen_shannon_divergence | 4.530661582946777 | 4.530661582946777 | 4.530661582946777 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4715566337108612 | 0.4715566337108612 | 0.4715566337108612 |
| pearson_corr | 0.6467952728271484 | 0.6467952728271484 | 0.6467952728271484 |
| jensen_shannon_divergence | 1.267215609550476 | 1.267215609550476 | 1.267215609550476 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4815959930419922 | 0.4815959930419922 | 0.4815959930419922 |
| pearson_corr | 0.6329220533370972 | 0.6329220533370972 | 0.6329220533370972 |
| jensen_shannon_divergence | 1.271597981452942 | 1.271597981452942 | 1.271597981452942 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7048315405845642 | 0.7048315405845642 | 0.7048315405845642 |
| pearson_corr | 0.46878334879875183 | 0.46878334879875183 | 0.46878334879875183 |
| jensen_shannon_divergence | 2.7436399459838867 | 2.7436399459838867 | 2.7436399459838867 |


----------------------------



### 0.01, tanh, relu

In [24]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='tanh')(in_layer)
        net = layers.Dense(64, activation='tanh')(net)
        net = layers.Dense(32, activation='tanh')(net)
        net = layers.Dense(16, activation='tanh')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='relu')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8393726944923401 | 0.8393726944923401 | 0.8393726944923401 |
| pearson_corr | 0.07707583904266357 | 0.07707583904266357 | 0.07707583904266357 |
| jensen_shannon_divergence | 4.9934539794921875 | 4.9934539794921875 | 4.9934539794921875 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5080324411392212 | 0.5080324411392212 | 0.5080324411392212 |
| pearson_corr | 0.6167089343070984 | 0.6167089343070984 | 0.6167089343070984 |
| jensen_shannon_divergence | 1.3230488300323486 | 1.3230488300323486 | 1.3230488300323486 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5095447897911072 | 0.5095447897911072 | 0.5095447897911072 |
| pearson_corr | 0.5905030965805054 | 0.5905030965805054 | 0.5905030965805054 |
| jensen_shannon_divergence | 1.4216761589050293 | 1.4216761589050293 | 1.4216761589050293 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7060512900352478 | 0.7060512900352478 | 0.7060512900352478 |
| pearson_corr | 0.47139427065849304 | 0.47139427065849304 | 0.47139427065849304 |
| jensen_shannon_divergence | 2.931382894515991 | 2.931382894515991 | 2.931382894515991 |


----------------------------



### 0.01, softmax, tanh

In [25]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='softmax')(in_layer)
        net = layers.Dense(64, activation='softmax')(net)
        net = layers.Dense(32, activation='softmax')(net)
        net = layers.Dense(16, activation='softmax')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='tanh')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4652860164642334 | 0.4652860164642334 | 0.4652860164642334 |
| pearson_corr | 0.6333311796188354 | 0.6333311796188354 | 0.6333311796188354 |
| jensen_shannon_divergence | 1.2057610750198364 | 1.2057610750198364 | 1.2057610750198364 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.48874586820602417 | 0.48874586820602417 | 0.48874586820602417 |
| pearson_corr | 0.615730881690979 | 0.615730881690979 | 0.615730881690979 |
| jensen_shannon_divergence | 1.3192119598388672 | 1.3192119598388672 | 1.3192119598388672 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4936138689517975 | 0.4936138689517975 | 0.4936138689517975 |
| pearson_corr | 0.6140884160995483 | 0.6140884160995483 | 0.6140884160995483 |
| jensen_shannon_divergence | 1.3392407894134521 | 1.3392407894134521 | 1.3392407894134521 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4698995351791382 | 0.4698995351791382 | 0.4698995351791382 |
| pearson_corr | 0.65058833360672 | 0.65058833360672 | 0.65058833360672 |
| jensen_shannon_divergence | 1.245864987373352 | 1.245864987373352 | 1.245864987373352 |


----------------------------



### 0.01, softmax, softmax

In [26]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='softmax')(in_layer)
        net = layers.Dense(64, activation='softmax')(net)
        net = layers.Dense(32, activation='softmax')(net)
        net = layers.Dense(16, activation='softmax')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='softmax')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.816419780254364 | 0.816419780254364 | 0.816419780254364 |
| pearson_corr | 0.09334118664264679 | 0.09334118664264679 | 0.09334118664264679 |
| jensen_shannon_divergence | 4.749266147613525 | 4.749266147613525 | 4.749266147613525 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.47420600056648254 | 0.47420600056648254 | 0.47420600056648254 |
| pearson_corr | 0.6392113566398621 | 0.6392113566398621 | 0.6392113566398621 |
| jensen_shannon_divergence | 1.2481073141098022 | 1.2481073141098022 | 1.2481073141098022 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4757595360279083 | 0.4757595360279083 | 0.4757595360279083 |
| pearson_corr | 0.6347171068191528 | 0.6347171068191528 | 0.6347171068191528 |
| jensen_shannon_divergence | 1.229459524154663 | 1.229459524154663 | 1.229459524154663 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6906036734580994 | 0.6906036734580994 | 0.6906036734580994 |
| pearson_corr | 0.47515103220939636 | 0.47515103220939636 | 0.47515103220939636 |
| jensen_shannon_divergence | 2.550014019012451 | 2.550014019012451 | 2.550014019012451 |


----------------------------



### 0.01, softmax, sigmoid

In [27]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='softmax')(in_layer)
        net = layers.Dense(64, activation='softmax')(net)
        net = layers.Dense(32, activation='softmax')(net)
        net = layers.Dense(16, activation='softmax')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='sigmoid')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7975616455078125 | 0.7975616455078125 | 0.7975616455078125 |
| pearson_corr | 0.10852620005607605 | 0.10852620005607605 | 0.10852620005607605 |
| jensen_shannon_divergence | 4.516049385070801 | 4.516049385070801 | 4.516049385070801 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4750479757785797 | 0.4750479757785797 | 0.4750479757785797 |
| pearson_corr | 0.6375152468681335 | 0.6375152468681335 | 0.6375152468681335 |
| jensen_shannon_divergence | 1.2491624355316162 | 1.2491624355316162 | 1.2491624355316162 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.47696611285209656 | 0.47696611285209656 | 0.47696611285209656 |
| pearson_corr | 0.6330741047859192 | 0.6330741047859192 | 0.6330741047859192 |
| jensen_shannon_divergence | 1.2279877662658691 | 1.2279877662658691 | 1.2279877662658691 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7111886143684387 | 0.7111886143684387 | 0.7111886143684387 |
| pearson_corr | 0.46393507719039917 | 0.46393507719039917 | 0.46393507719039917 |
| jensen_shannon_divergence | 2.7689945697784424 | 2.7689945697784424 | 2.7689945697784424 |


----------------------------



### 0.01, softmax, relu

In [28]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='softmax')(in_layer)
        net = layers.Dense(64, activation='softmax')(net)
        net = layers.Dense(32, activation='softmax')(net)
        net = layers.Dense(16, activation='softmax')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='relu')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8046173453330994 | 0.8046173453330994 | 0.8046173453330994 |
| pearson_corr | 0.10234614461660385 | 0.10234614461660385 | 0.10234614461660385 |
| jensen_shannon_divergence | 4.5911359786987305 | 4.5911359786987305 | 4.5911359786987305 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5421953797340393 | 0.5421953797340393 | 0.5421953797340393 |
| pearson_corr | 0.5111273527145386 | 0.5111273527145386 | 0.5111273527145386 |
| jensen_shannon_divergence | 1.5611237287521362 | 1.5611237287521362 | 1.5611237287521362 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7028366923332214 | 0.7028366923332214 | 0.7028366923332214 |
| pearson_corr | 0.23935218155384064 | 0.23935218155384064 | 0.23935218155384064 |
| jensen_shannon_divergence | 2.749696731567383 | 2.749696731567383 | 2.749696731567383 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.771818220615387 | 0.771818220615387 | 0.771818220615387 |
| pearson_corr | 0.10104642063379288 | 0.10104642063379288 | 0.10104642063379288 |
| jensen_shannon_divergence | 3.25044584274292 | 3.25044584274292 | 3.25044584274292 |


----------------------------



### 0.01, sigmoid, tanh

In [29]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='sigmoid')(in_layer)
        net = layers.Dense(64, activation='sigmoid')(net)
        net = layers.Dense(32, activation='sigmoid')(net)
        net = layers.Dense(16, activation='sigmoid')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='tanh')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.46112775802612305 | 0.46112775802612305 | 0.46112775802612305 |
| pearson_corr | 0.6442567706108093 | 0.6442567706108093 | 0.6442567706108093 |
| jensen_shannon_divergence | 1.1991392374038696 | 1.1991392374038696 | 1.1991392374038696 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4739938974380493 | 0.4739938974380493 | 0.4739938974380493 |
| pearson_corr | 0.647670567035675 | 0.647670567035675 | 0.647670567035675 |
| jensen_shannon_divergence | 1.2522757053375244 | 1.2522757053375244 | 1.2522757053375244 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4790405035018921 | 0.4790405035018921 | 0.4790405035018921 |
| pearson_corr | 0.6356619596481323 | 0.6356619596481323 | 0.6356619596481323 |
| jensen_shannon_divergence | 1.2580307722091675 | 1.2580307722091675 | 1.2580307722091675 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4579537510871887 | 0.4579537510871887 | 0.4579537510871887 |
| pearson_corr | 0.6658735871315002 | 0.6658735871315002 | 0.6658735871315002 |
| jensen_shannon_divergence | 1.174690842628479 | 1.174690842628479 | 1.174690842628479 |


----------------------------



### 0.01, sigmoid, softmax

In [30]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='sigmoid')(in_layer)
        net = layers.Dense(64, activation='sigmoid')(net)
        net = layers.Dense(32, activation='sigmoid')(net)
        net = layers.Dense(16, activation='sigmoid')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='softmax')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8078857660293579 | 0.8078857660293579 | 0.8078857660293579 |
| pearson_corr | 0.09990233927965164 | 0.09990233927965164 | 0.09990233927965164 |
| jensen_shannon_divergence | 4.649380683898926 | 4.649380683898926 | 4.649380683898926 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4534071385860443 | 0.4534071385860443 | 0.4534071385860443 |
| pearson_corr | 0.6666126251220703 | 0.6666126251220703 | 0.6666126251220703 |
| jensen_shannon_divergence | 1.1859110593795776 | 1.1859110593795776 | 1.1859110593795776 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.46555253863334656 | 0.46555253863334656 | 0.46555253863334656 |
| pearson_corr | 0.6493195295333862 | 0.6493195295333862 | 0.6493195295333862 |
| jensen_shannon_divergence | 1.1924610137939453 | 1.1924610137939453 | 1.1924610137939453 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6938168406486511 | 0.6938168406486511 | 0.6938168406486511 |
| pearson_corr | 0.47284844517707825 | 0.47284844517707825 | 0.47284844517707825 |
| jensen_shannon_divergence | 2.5917797088623047 | 2.5917797088623047 | 2.5917797088623047 |


----------------------------



### 0.01, sigmoid, sigmoid

In [31]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='sigmoid')(in_layer)
        net = layers.Dense(64, activation='sigmoid')(net)
        net = layers.Dense(32, activation='sigmoid')(net)
        net = layers.Dense(16, activation='sigmoid')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='sigmoid')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.8013951778411865 | 0.8013951778411865 | 0.8013951778411865 |
| pearson_corr | 0.10513327270746231 | 0.10513327270746231 | 0.10513327270746231 |
| jensen_shannon_divergence | 4.5590105056762695 | 4.5590105056762695 | 4.5590105056762695 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4672824740409851 | 0.4672824740409851 | 0.4672824740409851 |
| pearson_corr | 0.6516363024711609 | 0.6516363024711609 | 0.6516363024711609 |
| jensen_shannon_divergence | 1.2223061323165894 | 1.2223061323165894 | 1.2223061323165894 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.4714307188987732 | 0.4714307188987732 | 0.4714307188987732 |
| pearson_corr | 0.6429005861282349 | 0.6429005861282349 | 0.6429005861282349 |
| jensen_shannon_divergence | 1.215091347694397 | 1.215091347694397 | 1.215091347694397 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7065194249153137 | 0.7065194249153137 | 0.7065194249153137 |
| pearson_corr | 0.4681621789932251 | 0.4681621789932251 | 0.4681621789932251 |
| jensen_shannon_divergence | 2.737222194671631 | 2.737222194671631 | 2.737222194671631 |


----------------------------



### 0.01, sigmoid, relu

In [32]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='sigmoid')(in_layer)
        net = layers.Dense(64, activation='sigmoid')(net)
        net = layers.Dense(32, activation='sigmoid')(net)
        net = layers.Dense(16, activation='sigmoid')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='relu')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7912377119064331 | 0.7912377119064331 | 0.7912377119064331 |
| pearson_corr | 0.11416013538837433 | 0.11416013538837433 | 0.11416013538837433 |
| jensen_shannon_divergence | 4.395632266998291 | 4.395632266998291 | 4.395632266998291 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6281229257583618 | 0.6281229257583618 | 0.6281229257583618 |
| pearson_corr | 0.4122532904148102 | 0.4122532904148102 | 0.4122532904148102 |
| jensen_shannon_divergence | 1.83259117603302 | 1.83259117603302 | 1.83259117603302 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6638007760047913 | 0.6638007760047913 | 0.6638007760047913 |
| pearson_corr | 0.37537676095962524 | 0.37537676095962524 | 0.37537676095962524 |
| jensen_shannon_divergence | 2.5246992111206055 | 2.5246992111206055 | 2.5246992111206055 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.7193206548690796 | 0.7193206548690796 | 0.7193206548690796 |
| pearson_corr | 0.33011162281036377 | 0.33011162281036377 | 0.33011162281036377 |
| jensen_shannon_divergence | 3.1933071613311768 | 3.1933071613311768 | 3.1933071613311768 |


----------------------------



### 0.01, relu, tanh

In [33]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='relu')(in_layer)
        net = layers.Dense(64, activation='relu')(net)
        net = layers.Dense(32, activation='relu')(net)
        net = layers.Dense(16, activation='relu')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='tanh')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.9822139143943787 | 0.9822139143943787 | 0.9822139143943787 |
| pearson_corr | -0.006628669332712889 | -0.006628669332712889 | -0.006628669332712889 |
| jensen_shannon_divergence | 12.217270851135254 | 12.217270851135254 | 12.217270851135254 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.997576117515564 | 0.997576117515564 | 0.997576117515564 |
| pearson_corr | -0.001560187665745616 | -0.001560187665745616 | -0.001560187665745616 |
| jensen_shannon_divergence | 18.639245986938477 | 18.639245986938477 | 18.639245986938477 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.9980888366699219 | 0.9980888366699219 | 0.9980888366699219 |
| pearson_corr | -0.001557048293761909 | -0.001557048293761909 | -0.001557048293761909 |
| jensen_shannon_divergence | 27.838212966918945 | 27.838212966918945 | 27.838212966918945 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.9538848400115967 | 0.9538848400115967 | 0.9538848400115967 |
| pearson_corr | 0.002684287028387189 | 0.002684287028387189 | 0.002684287028387189 |
| jensen_shannon_divergence | 7.317476272583008 | 7.317476272583008 | 7.317476272583008 |


----------------------------



### 0.01, relu, softmax

In [34]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='relu')(in_layer)
        net = layers.Dense(64, activation='relu')(net)
        net = layers.Dense(32, activation='relu')(net)
        net = layers.Dense(16, activation='relu')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='softmax')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.9336927533149719 | 0.9336927533149719 | 0.9336927533149719 |
| pearson_corr | 0.03116733767092228 | 0.03116733767092228 | 0.03116733767092228 |
| jensen_shannon_divergence | 6.383246421813965 | 6.383246421813965 | 6.383246421813965 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.9554868936538696 | 0.9554868936538696 | 0.9554868936538696 |
| pearson_corr | 0.035315461456775665 | 0.035315461456775665 | 0.035315461456775665 |
| jensen_shannon_divergence | 7.905366897583008 | 7.905366897583008 | 7.905366897583008 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.9882022142410278 | 0.9882022142410278 | 0.9882022142410278 |
| pearson_corr | -0.005316429305821657 | -0.005316429305821657 | -0.005316429305821657 |
| jensen_shannon_divergence | 11.23586368560791 | 11.23586368560791 | 11.23586368560791 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.9039802551269531 | 0.9039802551269531 | 0.9039802551269531 |
| pearson_corr | 0.01759731024503708 | 0.01759731024503708 | 0.01759731024503708 |
| jensen_shannon_divergence | 6.785688877105713 | 6.785688877105713 | 6.785688877105713 |


----------------------------



### 0.01, relu, sigmoid

In [35]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='relu')(in_layer)
        net = layers.Dense(64, activation='relu')(net)
        net = layers.Dense(32, activation='relu')(net)
        net = layers.Dense(16, activation='relu')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='sigmoid')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.9841949939727783 | 0.9841949939727783 | 0.9841949939727783 |
| pearson_corr | -0.008075663819909096 | -0.008075663819909096 | -0.008075663819909096 |
| jensen_shannon_divergence | 10.69289779663086 | 10.69289779663086 | 10.69289779663086 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.9932752847671509 | 0.9932752847671509 | 0.9932752847671509 |
| pearson_corr | 0.007888782769441605 | 0.007888782769441605 | 0.007888782769441605 |
| jensen_shannon_divergence | 12.164841651916504 | 12.164841651916504 | 12.164841651916504 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.9920052289962769 | 0.9920052289962769 | 0.9920052289962769 |
| pearson_corr | -0.013737542554736137 | -0.013737542554736137 | -0.013737542554736137 |
| jensen_shannon_divergence | 16.570375442504883 | 16.570375442504883 | 16.570375442504883 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.9346619248390198 | 0.9346619248390198 | 0.9346619248390198 |
| pearson_corr | 0.06508958339691162 | 0.06508958339691162 | 0.06508958339691162 |
| jensen_shannon_divergence | 6.498879909515381 | 6.498879909515381 | 6.498879909515381 |


----------------------------



### 0.01, relu, relu

In [36]:
for g in combinations:
    #entrenamos el autoencoder
    print(g)
    print("===================\n")
    experiment_metrics, models, results = perform_experiment_2_mod(cv_folds=0, 
                        epochs=g[0], ##este es el que varia
                        batch_size=g[1], 
                        learning_rate=g[2], ##este es el que varia
                        optimizer=g[3],
                        learning_rate_scheduler=None,
                        input_transform=Percentage, #--> lo quitamos porque ya lo tenemos en relativo, con el 80 no
                        output_transform=tf.keras.layers.Softmax,
                        reconstruction_loss=MakeLoss(LossBrayCurtis, Percentage, None), 
                        latent_space=g[4], 
                        layers=g[5],
                        activation=g[6], 
                        activation_latent=g[7], 
                        data_microbioma_train=data_microbioma_train,
                        data_domain_train=data_domain_train,
                        show_results=False,#esto se ha cambiado para que no me muestre los resultados 
                        device='/CPU:0')
    
    #Guardamos el modelo, encoder y decoder
    model, encoder, _ ,decoder = models[0]
    
    
    def model_fn_latent():
        in_layer = layers.Input(shape=(data_domain_train.shape[1],))
        net = layers.Dense(128, activation='relu')(in_layer)
        net = layers.Dense(64, activation='relu')(net)
        net = layers.Dense(32, activation='relu')(net)
        net = layers.Dense(16, activation='relu')(net)
        out_layer = layers.Dense(latent_train.shape[1], activation='relu')(net) # 'tanh already'
        model = keras.Model(inputs=[in_layer], outputs=[out_layer], name='model')
        model.compile(optimizer=optimizers.Adam(lr=0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
        return model
    
    #funcion del modelado -- por ahora lo dejaremos asi, y voy a habalr con ella par aver que cambio, que tuneo
    latent_train = encoder.predict(data_microbioma_train)
    
    result_latent, model_latent = train_tl_noEnsemble(model_fn_latent,
                                                      latent_train,
                                                      latent_train,
                                                      data_domain_train,
                                                      data_domain_train,
                                                      epochs=100,
                                                      batch_size=16,
                                                      verbose=-1)

        #evaluamos el modelo
    latent_test = encoder.predict(data_microbioma_test)
        
    
        
    predictions = test_model_tl_noEnsemble(model_latent, decoder, Percentage, tf.keras.layers.Softmax, otu_columns, data_microbioma_test, data_domain_test)

    print("----------------------------\n")

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5907663106918335 | 0.5907663106918335 | 0.5907663106918335 |
| pearson_corr | 0.4308113753795624 | 0.4308113753795624 | 0.4308113753795624 |
| jensen_shannon_divergence | 1.6645249128341675 | 1.6645249128341675 | 1.6645249128341675 |


----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.50638347864151 | 0.50638347864151 | 0.50638347864151 |
| pearson_corr | 0.6213101744651794 | 0.6213101744651794 | 0.6213101744651794 |
| jensen_shannon_divergence | 1.290922999382019 | 1.290922999382019 | 1.290922999382019 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256], 'tanh', 'tanh']



...

----------------------------

[100, 64, 0.01, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 10, [512, 256], 'tanh', 'softmax']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.5209541320800781 | 0.5209541320800781 | 0.5209541320800781 |
| pearson_corr | 0.5863966345787048 | 0.5863966345787048 | 0.5863966345787048 |
| jensen_shannon_divergence | 1.314551830291748 | 1.314551830291748 | 1.314551830291748 |


----------------------------

[100, 64, 0.001, <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 15, [512, 256, 128], 'tanh', 'tanh']



## Test results 
| Metric           | Mean    | Min     | Max     |
|:-----------------|--------:|--------:|--------:|
| BrayCurtis | 0.6720988750457764 | 0.6720988750457764 | 0.6720988750457764 |
| pearson_corr | 0.3624771535396576 | 0.3624771535396576 | 0.3624771535396576 |
| jensen_shannon_divergence | 2.0583736896514893 | 2.0583736896514893 | 2.0583736896514893 |


----------------------------

